### Data collection

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import random


import scipy
import math
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split 
from scipy.sparse.linalg import svds 
from sklearn import preprocessing 

import nltk
from nltk.corpus import stopwords
from sklearn.preprocessing import normalize
nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer 


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lars\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
path='./Data/'

df_books=pd.read_csv(path +'filtered_books.csv') 
df_ratings=pd.read_csv(path + 'Ratings.csv') 
df_users=pd.read_csv(path + 'Users.csv') 

C:\Users\Lars\AppData\Local\Temp\ipykernel_25312\2624249305.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_books=pd.read_csv(path +'filtered_books.csv')


In [8]:
df_ratings.head()


,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [9]:
df_users.describe()



,User-ID,Age
count,278858.00000,168096.000000
mean,139429.50000,34.751434
std,80499.51502,14.428097
min,1.00000,0.000000
25%,69715.25000,24.000000
50%,139429.50000,32.000000
75%,209143.75000,44.000000
max,278858.00000,244.000000


### Preprocessing

In [ ]:
merged_df=pd.merge(df_users,df_ratings,on='User-ID')
merged_df=pd.merge(merged_df,df_books,on='ISBN')

merged_df.shape



(851511, 12)

### Collaberative Filtering

In [39]:
# Import necessary libraries
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from scipy.sparse import csr_matrix

# Load the dataset
data = pd.read_csv('data/ratings.csv', nrows=50000)

# Preprocess the data
data.fillna(0, inplace=True)

# Create a user-item matrix
user_item_matrix = data.pivot(index='User-ID', columns='ISBN', values='Book-Rating').fillna(0)

# Convert the user-item matrix to a sparse matrix
user_item_matrix_sparse = csr_matrix(user_item_matrix.values)

# Normalize the user-item matrix
scaler = StandardScaler(with_mean=False)
user_item_matrix_normalized = scaler.fit_transform(user_item_matrix_sparse)

# Compute user-user similarity
user_similarity = cosine_similarity(user_item_matrix_normalized)

# Convert the similarity matrix back to a DataFrame
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)

# Function to get top N similar users
def get_top_n_similar_users(user_id, n=5):
    similar_users = user_similarity_df[user_id].sort_values(ascending=False).head(n+1).index.tolist()
    similar_users.remove(user_id)
    return similar_users

# Function to generate recommendations for a user
def recommend_items(user_id, n=5):
    similar_users = get_top_n_similar_users(user_id, n)
    similar_users_ratings = user_item_matrix.loc[similar_users]
    recommended_items = similar_users_ratings.mean(axis=0).sort_values(ascending=False).head(n).index.tolist()
    return recommended_items

# Example usage
user_id = 10
recommendations = recommend_items(user_id, n=5)
print(f"Recommendations for user {user_id}: {recommendations}")

Recommendations for user 10: ['8804284749', '8804348879', '8807700786', '8811598680', '888113098X']


In [44]:
book_review_counts = df_ratings['ISBN'].value_counts()
popular_books = book_review_counts[book_review_counts >= 100].index
filtered_ratings = df_ratings[df_ratings['ISBN'].isin(popular_books)]


user_item_matrix = filtered_ratings.pivot(index='User-ID', columns='ISBN', values='Book-Rating').fillna(0)


sparse_matrix = csr_matrix(user_item_matrix.values)

item_similarity = cosine_similarity(sparse_matrix.T)  # Transpose for item-item similarity
item_similarity_df = pd.DataFrame(
    item_similarity, index=user_item_matrix.columns, columns=user_item_matrix.columns
)


item_similarity_df.sample(5, axis=1).sample(5, axis=0)  # Random subset to inspect

# Step 4: Define a function to predict ratings
def predict_ratings(user_id, user_item_matrix, item_similarity_df, k=5):
    user_ratings = user_item_matrix.loc[user_id]  # Ratings by the target user
    scores = item_similarity_df.dot(user_ratings).div(item_similarity_df.sum(axis=1))  # Weighted average
    scores[user_ratings > 0] = 0  # Ignore already rated items
    return scores.sort_values(ascending=False).head(k)  # Return top-k recommendations

# Display a sample of the item similarity matrix to the user
print("Item Similarity Matrix:")
print(item_similarity_df.head())
# Example: Predict top recommendations for a user
example_user_id = user_item_matrix.index[0]  # Choose the first user as an example
recommendations = predict_ratings(example_user_id, user_item_matrix, item_similarity_df, k=5)
recommendations


Item Similarity Matrix:
ISBN        002542730X  0060008032  0060096195  006016848X  0060173289  \
ISBN                                                                     
002542730X    1.000000    0.015558    0.000000    0.014222         0.0   
0060008032    0.015558    1.000000    0.036488    0.063930         0.0   
0060096195    0.000000    0.036488    1.000000    0.000000         0.0   
006016848X    0.014222    0.063930    0.000000    1.000000         0.0   
0060173289    0.000000    0.000000    0.000000    0.000000         1.0   

ISBN        0060175400  006019491X  0060199652  0060391626  0060392452  ...  \
ISBN                                                                    ...   
002542730X    0.024904    0.016457    0.013124    0.011569    0.025633  ...   
0060008032    0.024103    0.056502    0.027945    0.024634    0.000000  ...   
0060096195    0.015929    0.000000    0.000000    0.000000    0.022010  ...   
006016848X    0.022034    0.032032    0.025546    0.035030    

ISBN
0316899984    0.053442
0553282476    0.039898
080411918X    0.037778
038533334X    0.037013
0060916508    0.034303
dtype: float64